In [1]:
import pickle
import pandas as pd
import numpy as np
import scipy
import joblib

In [2]:
models_dict = joblib.load('models_dict.joblib')
models_dict.keys()

dict_keys(['Baseline', 'M-CURES'])

In [3]:
df_cohort = pd.read_csv('cohort.csv')
test_hosp, test_window, test_y = df_cohort['hosp_id'], df_cohort['window_id'], df_cohort['y']
cohort_IDs = df_cohort.set_index('ID')[[]]

In [4]:
df_cohort

,ID,hosp_id,window_id,y
0,90000-____0,90000,0,1
1,90000-____1,90000,1,1
2,90000-____2,90000,2,1
3,90001-____0,90001,0,0
4,90001-____1,90001,1,0


In [5]:
from sklearn import metrics, utils
from joblib import Parallel, delayed

def bootstrap_func(i, y_true, y_score):
    yte_true_b, yte_pred_b = utils.resample(y_true, y_score, replace=True, random_state=i)
    return metrics.roc_curve(yte_true_b, yte_pred_b), metrics.roc_auc_score(yte_true_b, yte_pred_b)

def get_roc_CI(y_true, y_score):
    roc_curves, auc_scores = zip(*Parallel(n_jobs=4)(delayed(bootstrap_func)(i, y_true, y_score) for i in range(1000)))
    print('Test AUC: ({:.3f}, {:.3f}) percentile 95% CI'.format(np.percentile(auc_scores, 2.5), np.percentile(auc_scores, 97.5)))

    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)
    for fpr, tpr, _ in roc_curves:
        tprs.append(np.interp(mean_fpr, fpr, tpr))
        tprs[-1][0] = 0.0
        aucs.append(metrics.auc(fpr, tpr))

    mean_tpr = np.mean(tprs, axis=0)
    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + 1.96 * std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - 1.96 * std_tpr, 0)
    return roc_curves, auc_scores, mean_fpr, tprs_lower, tprs_upper

def eval3():
    df_Yte = df_Yte_all.copy()
    df_Yte = df_Yte[df_Yte['window_id'] >= 1]
    df_Yte_agg = df_Yte.groupby(['hosp_id']).max()
    return df_Yte_agg

## Baseline model

In [6]:
baseline_clfs = models_dict['Baseline']
df_baseline = pd.read_csv('../preprocessing/sample_output/baseline.csv').set_index('ID')

In [7]:
df_baseline

,"age_value_(17.999, 47.0]","age_value_(47.0, 59.0]","age_value_(59.0, 67.0]","age_value_(67.0, 76.0]","age_value_(76.0, 102.0]",respiratoryrate_mask_0.0,respiratoryrate_mask_1.0,"respiratoryrate_max_(-0.001, 16.0]","respiratoryrate_max_(16.0, 18.0]","respiratoryrate_max_(18.0, 22.0]","respiratoryrate_max_(22.0, 200.0]","respiratoryrate_mean_(-0.001, 16.0]","respiratoryrate_mean_(16.0, 17.667]","respiratoryrate_mean_(17.667, 18.0]","respiratoryrate_mean_(18.0, 20.0]","respiratoryrate_mean_(20.0, 196.0]","respiratoryrate_min_(-0.001, 16.0]","respiratoryrate_min_(16.0, 18.0]","respiratoryrate_min_(18.0, 19.0]","respiratoryrate_min_(19.0, 196.0]","respiratoryrate_value_(-0.001, 16.0]","respiratoryrate_value_(16.0, 18.0]","respiratoryrate_value_(18.0, 20.0]","respiratoryrate_value_(20.0, 196.0]",spo2_mask_0.0,spo2_mask_1.0,"spo2_max_(-0.001, 94.0]","spo2_max_(94.0, 96.0]","spo2_max_(96.0, 97.0]","spo2_max_(97.0, 99.0]","spo2_max_(99.0, 100.0]","spo2_mean_(-0.001, 94.0]","spo2_mean_(94.0, 95.0]","spo2_mean_(95.0, 96.667]","spo2_mean_(96.667, 98.0]","spo2_mean_(98.0, 100.0]","spo2_min_(-0.001, 93.0]","spo2_min_(93.0, 95.0]","spo2_min_(95.0, 96.0]","spo2_min_(96.0, 98.0]","spo2_min_(98.0, 100.0]","spo2_value_(-0.001, 94.0]","spo2_value_(94.0, 95.0]","spo2_value_(95.0, 97.0]","spo2_value_(97.0, 98.0]","spo2_value_(98.0, 100.0]","_307928_value_(-0.001, 2.0]","_307928_value_(2.0, 3.0]","_307928_value_(3.0, 5.0]","_307928_value_(5.0, 22525.0]"
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
90000-____0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90000-____1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
90000-____2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
90001-____0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90001-____1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [8]:
eval_matrix = scipy.sparse.csr_matrix(cohort_IDs.join(df_baseline).values.astype(float))
all_y = np.array([clf.predict_proba(eval_matrix)[:,1] for clf in baseline_clfs])

y_scores = all_y.mean(0)
df_Yte_all = pd.DataFrame({'hosp_id': test_hosp, 'window_id': test_window, 'y': test_y, 'y_score': y_scores})
df_Yte_agg = eval3()
y_score = df_Yte_agg['y_score']
y_true = df_Yte_all.groupby(['hosp_id']).first()['y']
fpr, tpr, thresholds = metrics.roc_curve(y_true, y_score)
print('Test AUC: {:.3f}'.format(metrics.roc_auc_score(y_true, y_score)))

# Generate 95% CI
try:
    roc_curves, auc_scores, mean_fpr, tprs_lower, tprs_upper = get_roc_CI(y_true, y_score)
except:
    pass

Test AUC: 1.000


## M-CURES (lite) model

In [9]:
mcures_clfs = models_dict['M-CURES']
df_mcures = pd.read_csv('../preprocessing/sample_output/mcures.csv').set_index('ID')

In [10]:
df_mcures

,"age_value_(17.999, 47.0]","age_value_(47.0, 59.0]","age_value_(59.0, 67.0]","age_value_(67.0, 76.0]","age_value_(76.0, 102.0]",respiratoryrate_mask_0.0,respiratoryrate_mask_1.0,"respiratoryrate_max_(-0.001, 16.0]","respiratoryrate_max_(16.0, 18.0]","respiratoryrate_max_(18.0, 22.0]","respiratoryrate_max_(22.0, 200.0]","respiratoryrate_mean_(-0.001, 16.0]","respiratoryrate_mean_(16.0, 17.667]","respiratoryrate_mean_(17.667, 18.0]","respiratoryrate_mean_(18.0, 20.0]","respiratoryrate_mean_(20.0, 196.0]","respiratoryrate_min_(-0.001, 16.0]","respiratoryrate_min_(16.0, 18.0]","respiratoryrate_min_(18.0, 19.0]","respiratoryrate_min_(19.0, 196.0]","respiratoryrate_value_(-0.001, 16.0]","respiratoryrate_value_(16.0, 18.0]","respiratoryrate_value_(18.0, 20.0]","respiratoryrate_value_(20.0, 196.0]",spo2_mask_0.0,spo2_mask_1.0,"spo2_max_(-0.001, 94.0]","spo2_max_(94.0, 96.0]","spo2_max_(96.0, 97.0]","spo2_max_(97.0, 99.0]","spo2_max_(99.0, 100.0]","spo2_mean_(-0.001, 94.0]","spo2_mean_(94.0, 95.0]","spo2_mean_(95.0, 96.667]","spo2_mean_(96.667, 98.0]","spo2_mean_(98.0, 100.0]","spo2_min_(-0.001, 93.0]","spo2_min_(93.0, 95.0]","spo2_min_(95.0, 96.0]","spo2_min_(96.0, 98.0]","spo2_min_(98.0, 100.0]","spo2_value_(-0.001, 94.0]","spo2_value_(94.0, 95.0]","spo2_value_(95.0, 97.0]","spo2_value_(97.0, 98.0]","spo2_value_(98.0, 100.0]",81723_hilonormal_flag_value_H,81723_hilonormal_flag_value_HH,81723_hilonormal_flag_value_L,81723_hilonormal_flag_value_LL,81723_hilonormal_flag_value_N,81723_value_value_ <6.80,81723_value_value_ SEE BELOW,"81723_value_value_(6.7989999999999995, 7.32]","81723_value_value_(7.32, 7.36]","81723_value_value_(7.36, 7.4]","81723_value_value_(7.4, 7.43]","81723_value_value_(7.43, 7.73]",81799_hilonormal_flag_value_H,81799_hilonormal_flag_value_L,81799_hilonormal_flag_value_LL,81799_hilonormal_flag_value_N,81799_value_value_ <15,81799_value_value_ SEE BELOW,81799_value_value_ not available,"81799_value_value_(12.399000000000001, 23.0]","81799_value_value_(23.0, 26.0]","81799_value_value_(26.0, 30.0]","81799_value_value_(30.0, 36.0]","81799_value_value_(36.0, 59.0]","_307928_value_(-0.001, 2.0]","_307928_value_(2.0, 3.0]","_307928_value_(3.0, 5.0]","_307928_value_(5.0, 22525.0]",_313030_value_Continuous,_313030_value_Intermittent,_314689_value_Lying,_314689_value_Sitting,_314689_value_Standing,_355444_value_Developmentally supported position,_355444_value_HOB at 15 degrees,_355444_value_HOB at 30 degrees,_355444_value_HOB at 45 degrees,_355444_value_HOB at 60 degrees,_355444_value_HOB at 90 degrees,_355444_value_HOB flat (medical condition),_355444_value_Reverse Trendelenberg,_355444_value_other (see comments)
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
90000-____0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90000-____1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90000-____2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90001-____0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [11]:
eval_matrix = scipy.sparse.csr_matrix(cohort_IDs.join(df_mcures).values.astype(float))
all_y = np.array([clf.predict_proba(eval_matrix)[:,1] for clf in mcures_clfs])

y_scores = all_y.mean(0)
df_Yte_all = pd.DataFrame({'hosp_id': test_hosp, 'window_id': test_window, 'y': test_y, 'y_score': y_scores})
df_Yte_agg = eval3()
y_score = df_Yte_agg['y_score']
y_true = df_Yte_all.groupby(['hosp_id']).first()['y']
fpr, tpr, thresholds = metrics.roc_curve(y_true, y_score)
print('Test AUC: {:.3f}'.format(metrics.roc_auc_score(y_true, y_score)))

# Generate 95% CI
try:
    roc_curves, auc_scores, mean_fpr, tprs_lower, tprs_upper = get_roc_CI(y_true, y_score)
except:
    pass

Test AUC: 1.000
